In [1]:
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving spam.csv to spam.csv


In [2]:
import numpy as np
import pandas as pd

### 파일 읽어오기

In [17]:
df = pd.read_csv(filename, encoding='ISO-8859-1')
df.head(3)

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN


In [18]:
del df['Unnamed: 2']
del df['Unnamed: 3']
del df['Unnamed: 4']

In [19]:
df['label'] = df.v1.apply(lambda x: 1 if x == 'spam' else 0)

In [20]:
# 다른 이름으로 변경 
df=df.rename({'v1':'class','v2':'message'},axis=1)
df.head()

,class,message,label
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   class    5572 non-null   object
 1   message  5572 non-null   object
 2   label    5572 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


In [22]:
# spam 과 ham의 value 
df['class'].value_counts()

ham     4825
spam     747
Name: class, dtype: int64

In [23]:
df.message[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

### Text 전처리

In [24]:
# 중복 확인
df.message.nunique()

5169

In [25]:
# 중복 데이터 제거
df.drop_duplicates('message', keep='first', inplace=True)

In [26]:
# 정규식을 사용해서 영어 이외읭 문자는 공백으로 변환
df['message'] = df.message.str.replace('[^A-Za-z ]', '')
df.message[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [28]:
df['class'].value_counts()

ham     4516
spam     653
Name: class, dtype: int64

## 2. 학습 및 테스트 데이터 셋으로 분리

In [29]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.message, df['class'], test_size = 0.2,
    stratify = df['class'], random_state=2021
)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4135,), (1034,), (4135,), (1034,))

### 3. 텍스트 변환

In [30]:
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer

##### CountVectorizer, ngram_range(1,1)

In [31]:
cvect1 = CountVectorizer(stop_words='english')
cvect1.fit(X_train)
X_train_cv1 = cvect1.transform(X_train)
X_test_cv1 = cvect1.transform(X_test)

##### CountVectorizer, ngram_range(1,2)

In [ ]:
cvect2 = CountVectorizer(stop_words='english', ngram_range =(1,2))
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
X_test_cv2 = cvect2.transform(X_test)

##### TfidfVectorizer, ngram_range(1,1)

In [ ]:
tvect1 = TfidfVectorizer(stop_words='english')
tvect1.fit(X_train)
X_train_tv1 = tvect1.transform(X_train)
X_test_tv1 = tvect1.transform(X_test)
X_train_tv1.shape, X_test_tv1.shape

((4457, 6589), (1115, 6589))

##### TfidfVectorizer, ngram_range(1,2)

In [ ]:
tvect2 = TfidfVectorizer(stop_words ='english', ngram_range=(1,2))
tvect2.fit(X_train)
X_train_tv2 = tvect2.transform(X_train)
X_test_tv2 = tvect2.transform(X_test)

### 4. 로지스틱 회귀 모델로 학습(훈련)

In [33]:
from sklearn.linear_model import LogisticRegression

In [34]:
# 객체 생성 및 훈련 
lr1 = LogisticRegression()
%time lr1.fit(X_train_cv1, y_train)

CPU times: user 60.6 ms, sys: 1.91 ms, total: 62.5 ms
Wall time: 74.2 ms


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr2 = LogisticRegression()
lr2.fit(X_train_cv2, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr3 = LogisticRegression()
lr3.fit(X_train_tv1, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [ ]:
lr4= LogisticRegression()
lr4.fit(X_train_tv2, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
# 테스트 데이터에 대해 예측 - 테스트 테이터의 X만 필요
pred1 = lr1.predict(X_test_cv1)


In [ ]:
pred2 = lr2.predict(X_test_cv2)

In [ ]:
pred3 = lr3.predict(X_test_tv1)

In [ ]:
pred4 = lr4.predict(X_test_tv2)

### 5. 평가

In [36]:
from sklearn.metrics import accuracy_score

In [37]:
score1= accuracy_score(y_test, pred1)
score1

0.9777562862669246

In [ ]:
score2 = accuracy_score(y_test, pred2)
score2

0.9748878923766816

In [ ]:
score3 = accuracy_score(y_test, pred3)
score3

0.95695067264574

In [ ]:
score4 = accuracy_score(y_test, pred4)
score4

0.9515695067264573

결과 : 카운터 백터, ngram_range(1,1)가장 정확하다

#### 실제 테스트

In [38]:
text = ['love has one law  make happy the person you love  in the same way friendship has one law  never make ur friend feel alone until you are alive     gud night']

In [39]:
spam_cv = cvect1.transform(text)
pred = lr1.predict(spam_cv)
pred

array(['ham'], dtype=object)